# Thinking through data access

We want to read a large number of netCDF files as a single dataset, and then analyze that. How do we think about it?

## First create a list of files

The `glob` package is good for this ([docs](https://docs.python.org/3/library/glob.html))

> The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. No tilde expansion is done, but *, ?, and character ranges expressed with [] will be correctly matched. 

Two important pieces:
-  The tilde `~` is not expanded to the user's home directory. Use `os.path.expanduser` for that.
- The list of files is not sorted ! Use `sorted` to sort the list.

Here's a list of files: these are timeseries files with output for years 1850-2100, and 50 ensemble members.

In [1]:
import glob

files = glob.glob('/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/*smbb**h1**18500101-21001231*')
files

['/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1251.011.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1281.019.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1071.004.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1171.009.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1151.008.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1231.018.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE/PRECT/ENSEMBLE/b.e21.BHISTsmbb.f09_g17.LE2-1301.014.cam.h1.PRECT.18500101-21001231.nc',
 '/glade/scratch/anukesh/CESM-LE

There are 50 files, one per ensemble member

In [2]:
len(files)

50

## Open a single file

In [3]:
import xarray as xr

single = xr.open_dataset(files[0])
single

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, lev: 32, ilev: 33, time: 91617, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ...            ...
    n2ovmr        (time) float64 ...
    f11vmr        (time) float64 ...
    f12vmr        (time) float64 ...
    sol_tsi       (time) float64 ...
    nsteph        (time) int32 ...
    PRECT         (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1251.011
    logname:           sunseon
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1251-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:42 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

First check data size

In [4]:
single.nbytes / 1e9  # approx GB

20.274116128

Each single file is 20GB and we have 50 of them, so approximately a terabyte in total. We will have to use dask.

That means we need to make chunking decisions.

Later on, we will extract time series at a single point, so let's chunk in space, and choosing chunksizes for the data variable `PRECT`.

Start by looking at dimension names for `PRECT`

In [5]:
single.PRECT

<xarray.DataArray 'PRECT' (time: 91617, lat: 192, lon: 288)>
[5066053632 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

### Choosing a chunk size

This is a timeseries file with daily average output using the `noleap` calendar.

We will concatenate ensemble members together to create a single dataset along a new dimension `"ensemble"`. Today, we *cannot* create an xarray dataset with chunksizes that span files. In other words, because there is one file per ensemble member, and we are concatenating ensemble members along a new dimension, the chunksize for the new dimension **will** be one. It is possible to rechunk later, but that will involve expensive communication that is best to avoid unless you really need to do so.

We have decided to chunk along space because we want to plot time series at a single point later. After some experimenting we choose a size of 16 along `lat`, 32 along `lon`, and all timesteps in a single chunk, for a chunksize of ~180MB.

```{tip}
Many other chunking choices are possible, it all depends on what you want to do later.  For example we could have bigger spatial chunks, and smaller chunks along time. Here is some reading material on chunking/xarray/dask:
- https://docs.dask.org/en/stable/array-best-practices.html#select-a-good-chunk-size
- https://docs.xarray.dev/en/stable/user-guide/dask.html#optimization-tips
- https://docs.dask.org/en/latest/array-chunks.html
- https://blog.dask.org/2020/07/30/beginners-config
```

> When choosing the size of chunks it is best to make them neither too small, nor too big (around 100MB is often reasonable). Each chunk needs to be able to fit into the worker memory and operations on that chunk should take some non-trivial amount of time (more than 100ms). For many more recommendations take a look at the docs on [chunks](https://docs.dask.org/en/latest/array-chunks.html)...




In [6]:
single.PRECT.chunk({"lat": 16, "lon": 32})

<xarray.DataArray 'PRECT' (time: 91617, lat: 192, lon: 288)>
dask.array<xarray-<this-array>, shape=(91617, 192, 288), dtype=float32, chunksize=(91617, 16, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

#### Choosing combine options

Let's trying reading just 3 files to make sure the output looks as we expect

In [7]:
xr.open_mfdataset(
    # make sure we sort
    sorted(files[:3]),
    # concatenate along a new dimension called "ensemble"
    concat_dim='ensemble',
    # jsut concatenate them together
    combine='nested',
    chunks={"lat": 16, "lon": 32},
    parallel=True,
)

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, ensemble: 3, lev: 32, ilev: 33,
                   time: 91617, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: ensemble, nbnd
Data variables: (12/26)
    gw            (ensemble, lat) float64 dask.array<chunksize=(1, 16), meta=np.ndarray>
    hyam          (ensemble, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    hybm          (ensemble, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    P0            (ensemble) float64 1e+05 1e+05 1e+05
    hyai          (ensemble, ilev) float64 dask.array<chunksize=(1, 33), meta=np.ndarray>
    hybi          (ensemble, ilev) float64 dask.array<chunksize=(1, 33), meta=np.ndarray>
    ...            ...
    n2ovmr        (ensemble, time) float64 dask.array<chunksize=(1, 91617), meta=np.ndarray>
    f11vmr        (ensemble, time) float64 dask.array<chunksize=(1, 91617), meta=np.ndarray>
    f12vmr        (ensemble, time) float64 dask.array<chunksize=(1, 91617), meta=np.ndarray>
    sol_tsi       (ensemble, time) float64 dask.array<chunksize=(1, 91617), meta=np.ndarray>
    nsteph        (ensemble, time) int32 dask.array<chunksize=(1, 91617), meta=np.ndarray>
    PRECT         (ensemble, time, lat, lon) float32 dask.array<chunksize=(1, 91617, 16, 32), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1251.011
    logname:           sunseon
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1251-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:42 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

Notice that *all* variables have been concatenated along the `ensemble` dimension even if we know it to be a constant: e.g. `P0`.

Xarray has a number of options to control this concatenation behaviour. The [normal recommendation](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) is the hard-to-interpret sequence `data_vars="minimal", coords="minimal", compat="override"`. What does this mean?
1. `"minimal"` for `data_vars` and `coords` means only concatenate variables that have the concatenation dimension already.
2. For those variables without the concatenation dimension, xarray look at the `compat` kwarg. For `compat="different"`, the default, Xarray will check for equality of the variable across all files. Those that are different get concatenated, those that are the same, are simply copied over. This can get quite expensive, so `compat="override"` allows you to skip equality checking and simply pick the variable from the first file. This is great for so-called 'static variables' such as grid variables that are invariant in time (and ensemble member).

Let's try that

In [8]:
combined = xr.open_mfdataset(
    # make sure we sort
    sorted(files[:3]),
    # concatenate along a new dimension called "ensemble"
    concat_dim='ensemble',
    chunks={"lat": 16, "lon": 32},
    data_vars="minimal",
    coords="minimal",
    compat="override",
    # just concatenate them together
    combine='nested',
    parallel=True,
)
combined

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, lev: 32, ilev: 33, time: 91617, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (lat) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    hyam          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    hybm          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    P0            float64 ...
    hyai          (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    hybi          (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f11vmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f12vmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(91617,), meta=np.ndarray>
    PRECT         (time, lat, lon) float32 dask.array<chunksize=(91617, 16, 32), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1251.011
    logname:           sunseon
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1251-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:42 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

Oops this doesn't work for us! We didn't concatenate `PRECT` along the new `ensemble` dimension.

In [9]:
combined.PRECT.dims

('time', 'lat', 'lon')

Our dataset doesn't really fit the assumptions of `open_mfdataset`. Luckily we can modify our datasets before the concatenation stage using the `preprocess` kwarg ([docs](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset))
>  `preprocess`: If provided, call this function on each dataset prior to concatenation. You can find the file-name from which each dataset was loaded in ds.encoding["source"].

What we'll do is to add a new dimension `ensemble` to the `PRECT` variable using [`expand_dims`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.expand_dims.html#xarray.DataArray.expand_dims)

In [10]:
def add_ensemble_dim(ds):
    ds["PRECT"] = ds.PRECT.expand_dims('ensemble')
    return ds


combined = xr.open_mfdataset(
    # make sure we sort
    sorted(files[:3]),
    # chunk the dataset from each file properly
    chunks={"lat": 16, "lon": 32},
    # concatenate along a new dimension called "ensemble"
    concat_dim='ensemble',
    data_vars="minimal",
    coords="minimal",
    compat="override",
    combine='nested',
    parallel=True,
    preprocess=add_ensemble_dim,
)
combined

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, lev: 32, ilev: 33, time: 91617, nbnd: 2,
                   ensemble: 3)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: nbnd, ensemble
Data variables: (12/26)
    gw            (lat) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    hyam          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    hybm          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    P0            float64 ...
    hyai          (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    hybi          (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f11vmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f12vmr        (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(91617,), meta=np.ndarray>
    PRECT         (ensemble, time, lat, lon) float32 dask.array<chunksize=(1, 91617, 16, 32), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1251.011
    logname:           sunseon
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1251-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:42 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

Much better!

In [11]:
combined.PRECT

<xarray.DataArray 'PRECT' (ensemble: 3, time: 91617, lat: 192, lon: 288)>
dask.array<concatenate, shape=(3, 91617, 192, 288), dtype=float32, chunksize=(1, 91617, 16, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: ensemble
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

## Create a dask cluster

We'll use an adaptive cluster to be polite

In [12]:
import ncar_jobqueue

cluster = ncar_jobqueue.NCARCluster(local_directory="/local_scratch/pbs.$PBS_JOBID/dask/spill")
# create an adaptive cluster with one job always requested,
# scale to a maximum of 6 jobs
# and hold on to each job for 600 seconds of idle time
cluster.adapt(minimum_jobs=1, maximum_jobs=6, wait_count=600)

/glade/u/home/dcherian/miniconda3/envs/pump/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Connect a "Client" to the cluster

In [16]:
from distributed import Client

client = Client()
client

/glade/u/home/dcherian/miniconda3/envs/pump/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37385 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/37385/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/37385/status,Workers: 2
Total threads: 2,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43040,Workers: 2
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/37385/status,Total threads: 2
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:45314,Total threads: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/42995/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:36384,


## Read and concatenate multiple data files using xarray.open_mfdataset 

Now we can scale it up. We generalize a little by having `add_ensemble_dim` expand the dimensions of any variable with 3 or more dimensions.


In [14]:
def add_ensemble_dim(ds):
    # find all 3D variables
    names = [name for name, variable in ds.variables.items() if variable.ndim >= 3]
    # add a new dimension `ensemble` of size 1
    # and replace the existing 3D variables.
    ds.update(ds[names].expand_dims('ensemble'))
    return ds


combined = xr.open_mfdataset(
    # make sure we sort
    sorted(files),
    # chunk each individual file
    chunks={"lat": 16, "lon": 32},
    # Add the ensemble dimension to 3D variables
    preprocess=add_ensemble_dim,
    # concatenate along a new dimension called "ensemble"
    concat_dim='ensemble',
    # only concatenate variables with the `ensemble` dimension.
    data_vars="minimal",
    coords="minimal",
    compat="override",
    combine='nested',
    parallel=True,
)
combined

<xarray.Dataset>
Dimensions:              (lat: 192, lon: 288, lev: 32, ilev: 33, cosp_prs: 7,
                          nbnd: 2, cosp_tau: 7, cosp_scol: 10, cosp_ht: 40,
                          cosp_sr: 15, cosp_sza: 5, cosp_dbze: 15,
                          cosp_htmisr: 16, cosp_tau_modis: 7, cosp_reffice: 6,
                          cosp_reffliq: 6, time: 91617, ensemble: 50)
Coordinates: (12/16)
  * lat                  (lat) float64 -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon                  (lon) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * lev                  (lev) float64 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * ilev                 (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * cosp_prs             (cosp_prs) float64 9e+04 7.4e+04 ... 2.45e+04 9e+03
  * cosp_tau             (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 100.0
    ...                   ...
  * cosp_dbze            (cosp_dbze) float64 -72.5 -42.5 -37.5 ... 17.5 50.0
  * cosp_htmisr          (cosp_htmisr) float64 0.0 250.0 ... 1.6e+04 1.8e+04
  * cosp_tau_modis       (cosp_tau_modis) float64 0.15 0.8 2.45 ... 41.5 100.0
  * cosp_reffice         (cosp_reffice) float64 5e-06 1.5e-05 ... 5e-05 7.5e-05
  * cosp_reffliq         (cosp_reffliq) float64 4e-06 9e-06 ... 1.75e-05 2.5e-05
  * time                 (time) object 1850-01-01 00:00:00 ... 2101-01-01 00:...
Dimensions without coordinates: nbnd, ensemble
Data variables: (12/35)
    gw                   (lat) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    hyam                 (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    hybm                 (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    P0                   float64 ...
    hyai                 (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    hybi                 (ilev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    ...                   ...
    n2ovmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f11vmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f12vmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    sol_tsi              (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    nsteph               (time) int32 dask.array<chunksize=(91617,), meta=np.ndarray>
    PRECT                (ensemble, time, lat, lon) float32 dask.array<chunksize=(1, 91617, 16, 32), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1011.001
    logname:           sunseon
    host:              mom2
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1011-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:41 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

In [18]:
def add_coords(ds):
    # find all 3D variables
    names = [name for name, variable in ds.variables.items() if variable.ndim <= 2]
    return ds.set_coords(names)


combined = xr.open_mfdataset(
    # make sure we sort
    sorted(files),
    # chunk each individual file
    chunks={"lat": 16, "lon": 32},
    # Add the ensemble dimension to 3D variables
    preprocess=add_coords,
    # concatenate along a new dimension called "ensemble"
    concat_dim='ensemble',
    # only concatenate variables with the `ensemble` dimension.
    data_vars="all",
    coords="minimal",
    compat="override",
    combine='nested',
    parallel=True,
)
combined

2023-05-15 13:29:03,525 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:04,165 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:04,930 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:05,666 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:06,475 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:07,329 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:08,285 - distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
2023-05-15 13:29:09,251 - distributed.utils_perf - WARNING - full garbage collections took

<xarray.Dataset>
Dimensions:              (lat: 192, lon: 288, lev: 32, ilev: 33, cosp_prs: 7,
                          nbnd: 2, cosp_tau: 7, cosp_scol: 10, cosp_ht: 40,
                          cosp_sr: 15, cosp_sza: 5, cosp_dbze: 15,
                          cosp_htmisr: 16, cosp_tau_modis: 7, cosp_reffice: 6,
                          cosp_reffliq: 6, time: 91617, ensemble: 50)
Coordinates: (12/50)
  * lat                  (lat) float64 -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon                  (lon) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    gw                   (lat) float64 dask.array<chunksize=(16,), meta=np.ndarray>
  * lev                  (lev) float64 3.643 7.595 14.36 ... 957.5 976.3 992.6
    hyam                 (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    hybm                 (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    ...                   ...
    ch4vmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    n2ovmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f11vmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    f12vmr               (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    sol_tsi              (time) float64 dask.array<chunksize=(91617,), meta=np.ndarray>
    nsteph               (time) int32 dask.array<chunksize=(91617,), meta=np.ndarray>
Dimensions without coordinates: nbnd, ensemble
Data variables:
    PRECT                (ensemble, time, lat, lon) float32 dask.array<chunksize=(1, 91617, 16, 32), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1011.001
    logname:           sunseon
    host:              mom2
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1011-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1
    history:           Sat Apr 29 19:51:41 2023: ncrcat b.e21.BHISTsmbb.f09_g...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

In [ ]:
ds_emean = combined.mean(dim='ensemble')
ds_emean.PRECT.sel(lon=270, lat=37.22, method='nearest').plot()